In [1]:
from Strategies.StrategyBB import StrategyBB
from Strategies.StrategyMACD import StrategyMACD
from Strategies.StrategyRSI import StrategyRSI
from historyData.HistoryData import HistoryData
from tinkoff.invest.grpc.marketdata_pb2 import CandleInterval
from datetime import datetime

import asyncio

In [2]:
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN
data_set = HistoryData()

In [3]:
candles_down = None
candles_up = None
candles_flat = None

async def data_get():
    global candles_down, candles_up, candles_flat
    
    candles_down = await (data_set.get_tinkoff_server_data(datetime(2022, 2, 1), datetime(2022, 3, 1), candle_interval))
    candles_up = await (data_set.get_tinkoff_server_data(datetime(2023, 3, 1), datetime(2023, 4, 1), candle_interval))
    candles_flat = await (data_set.get_tinkoff_server_data(datetime(2024, 1, 1), datetime(2024, 2, 1), candle_interval))

# Run the event loop directly using await
await data_get()

# Creates a Data Set For Future Model

In [4]:
all_candles = [candles_down, candles_up, candles_flat]
len(all_candles[1])

22663

In [5]:
from AppLogic.Settings import Settings
from tinkoff.invest.utils import quotation_to_decimal

commission = 0.0005
all_data_index_buy_sell = []

for candles in all_candles:
    data_index_buy_sell = []
    total_earn = 0
    total_sum_earned = 0
    bought_at = -1
    for i, candle in enumerate(list(candles)):
        if i == 0:
            continue
        
        current_price = float(quotation_to_decimal(candle.close))
        if current_price > float(quotation_to_decimal(candles[i - 1].close)):
            if len(data_index_buy_sell) == 0 or list(data_index_buy_sell[-1].keys())[0] == "sold":
                bought_at = float(quotation_to_decimal(candles[i - 1].close))
                data_index_buy_sell.append({"bought": i - 1})
            total_earn += current_price - float(quotation_to_decimal(candles[i - 1].close))
        elif current_price < float(quotation_to_decimal(candles[i - 1].close)):
            data_index_buy_sell.append({"sold": i - 1})
            if total_earn > 0.1 / 100 * bought_at:
                total_sum_earned += total_earn
            total_earn = 0
            
    all_data_index_buy_sell.append(data_index_buy_sell)
    print(f"Possible profit: {total_sum_earned}", f"Length of the marked data: {len(data_index_buy_sell)}")

Possible profit: 2350.619999999995 Length of the marked data: 12699
Possible profit: 1723.6899999999978 Length of the marked data: 15301
Possible profit: 462.9800000000014 Length of the marked data: 13520


In [15]:
import pandas as pd

all_new_data_buy = list()
for data_index_buy_sell in all_data_index_buy_sell:
    new_data_buy = list()
    for i, data in enumerate(data_index_buy_sell):
        if list(data_index_buy_sell[i].keys())[0] != "sold":
            new_data_buy.append(data)
    all_new_data_buy.append(new_data_buy)

all_df_data = list()

for c_i, candles in enumerate(all_candles):
    df_data = {"index": [],
               "result": []}
    buy_counter = 0
    for i, candle in enumerate(list(candles)):
        df_data["index"].append(i)
        
        if i == all_new_data_buy[c_i][buy_counter]["bought"]:
            df_data["result"].append(1)
            buy_counter += 1 if buy_counter < len(all_new_data_buy[c_i]) - 1 else 0
        else:
            df_data["result"].append(0)
    all_df_data.append(df_data)
        
print(f"All different period data len: {len(all_df_data)}")

All different period data len: 3


# Add Strategy Signals To DF

In [16]:
from Strategies.StrategyEMA import StrategyEMA

# candle interval init (for this case is always 1 min)
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN

# init all strategies
EMA_down = StrategyEMA()
EMA_up = StrategyEMA()
EMA_flat = StrategyEMA()

RSI_down = StrategyRSI()
RSI_up = StrategyRSI()
RSI_flat = StrategyRSI()

MACD_down = StrategyMACD()
MACD_up = StrategyMACD()
MACD_flat = StrategyMACD()

BB_down = StrategyBB()
BB_up = StrategyBB()
BB_flat = StrategyBB()

strategies_blocks = [[EMA_down, EMA_up, EMA_flat], 
                    [RSI_down, RSI_up, RSI_flat], 
                    [MACD_down, MACD_up, MACD_flat], 
                    [BB_down, BB_up, BB_flat]]

for strategy_block in strategies_blocks:
    strategy_block[0].initialize_moving_avg_container(candles_down[:strategy_block[0].history_candles_length])
    strategy_block[1].initialize_moving_avg_container(candles_up[:strategy_block[1].history_candles_length])
    strategy_block[2].initialize_moving_avg_container(candles_flat[:strategy_block[2].history_candles_length])

print(f"Strategy blocks len: {len(strategies_blocks)}")     

Strategy blocks len: 4


In [17]:
from Strategies import StrategyABS

all_df_data_st = [{"past_EMA": [],
                "current_EMA": [],
                "current_RSI": [],
                "past_RSI": [],
                "past_MACD": [],
                "current_MACD": [],
                "upper_BB": [],
                "lower_BB": []} for i in range(len(strategies_blocks[0]))]

def add_strategy_params(strategy_name: str, candle_index: int, history_candles_length: int, strategy: StrategyABS, current_layer: int) -> None:
            global all_df_data_st
            past = "past_" + strategy_name
            current = "current_" + strategy_name
            
            if candle_index == history_candles_length:
                [all_df_data_st[current_layer][past].append(None) for i in range(strategy.history_candles_length)]
                [all_df_data_st[current_layer][current].append(None) for i in range(strategy.history_candles_length)]
            params = strategy.get_candle_param(all_candles[str_i][candle_index])
            all_df_data_st[current_layer][past].append(params[0])
            all_df_data_st[current_layer][current].append(params[1])

for strategy_block in strategies_blocks:
    for str_i, strategy_type in enumerate(strategy_block):
        for candle_index in range(strategy_type.history_candles_length, len(all_candles[str_i])):
            if strategy_type.__class__.__name__ == "StrategyEMA":
                add_strategy_params("EMA", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyRSI":
                add_strategy_params("RSI", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyMACD":
                add_strategy_params("MACD", candle_index, strategy_type.history_candles_length, strategy_type, str_i)
            elif strategy_type.__class__.__name__ == "StrategyBB":
                if candle_index == strategy_type.history_candles_length:
                    [all_df_data_st[str_i]["upper_BB"].append(None) for i in range(strategy_type.history_candles_length)]
                    [all_df_data_st[str_i]["lower_BB"].append(None) for i in range(strategy_type.history_candles_length)]
                c_params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
                all_df_data_st[str_i]["upper_BB"].append(c_params[1])
                all_df_data_st[str_i]["lower_BB"].append(c_params[2])

print(f"Small length check should be the length of candles: {len(all_df_data_st[2]["lower_BB"])}, candle len: {len(all_candles[2])}")

Small length check should be the length of candles: 21769, candle len: 21769


In [60]:
# convert to df
for i in range(len(all_df_data)):
    all_df_data[i] = pd.DataFrame(all_df_data[i])

for i in range(len(all_df_data)):
    for name in all_df_data_st[i]:
        all_df_data[i][name] = all_df_data_st[i][name]
 
biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length

# delete first NaN elements
for i, df in enumerate(all_df_data):
    all_df_data[i] = df.drop(df.index[:biggest_gap])

# concatenate all DF
concatenated_df = pd.DataFrame()
for data in all_df_data:
    concatenated_df = pd.concat([concatenated_df, data], ignore_index=True)
    
concatenated_df

,index,result,past_EMA,current_EMA,past_RSI,current_RSI,past_MACD,current_MACD,upper_BB,lower_BB
0,200,0,-1.209035,-12.677693,53.919357,93.194427,-0.233084,-0.194889,271.441659,263.052341
1,201,0,-12.677693,-22.862078,93.194427,92.658673,-0.194889,-0.211452,271.110899,262.786101
2,202,1,-22.862078,-31.847178,92.658673,91.852726,-0.211452,-0.290344,270.537024,262.514976
3,203,0,-31.847178,-39.936275,91.852726,91.896176,-0.290344,-0.263743,269.931718,262.442282
4,204,0,-39.936275,-47.213396,91.896176,91.938554,-0.263743,-0.173091,269.336210,262.511790
...,...,...,...,...,...,...,...,...,...,...
61205,21764,0,-0.077712,-0.074969,51.094821,51.333664,-0.010906,-0.009423,276.205463,276.010537
61206,21765,1,-0.074969,-0.071588,51.333664,51.081684,-0.009423,-0.008656,276.206069,276.007931
61207,21766,0,-0.071588,-0.071903,51.081684,52.032965,-0.008656,-0.005167,276.206108,276.010892
61208,21767,0,-0.071903,-0.072117,52.032965,52.032965,-0.005167,-0.002696,276.206026,276.013974


In [18]:
# 1 DF usage
index = 0

all_df_data[index] = pd.DataFrame(all_df_data[index])

for name in all_df_data_st[index]:
    all_df_data[index][name] = all_df_data_st[index][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length
            
all_df_data[index] = all_df_data[index].drop(all_df_data[index].index[:biggest_gap])

concatenated_df = all_df_data[index]
concatenated_df

,index,result,past_EMA,current_EMA,current_RSI,past_RSI,past_MACD,current_MACD,upper_BB,lower_BB
200,200,0,270.611014,267.201857,47.195712,47.078772,-0.931734,-0.736940,271.441659,263.052341
201,201,0,270.547422,266.917871,46.418695,47.195712,-1.001146,-0.789781,271.110899,262.786101
202,202,1,270.470930,266.531407,45.280853,46.418695,-1.152609,-0.862347,270.537024,262.514976
203,203,0,270.403458,266.260797,46.095365,45.280853,-1.191931,-0.928264,269.931718,262.442282
204,204,0,270.344717,266.093102,46.874850,46.095365,-1.144540,-0.971519,269.336210,262.511790
...,...,...,...,...,...,...,...,...,...,...
17373,17373,0,129.228707,130.238397,53.147416,53.147416,0.242951,0.245011,130.764206,129.736794
17374,17374,0,129.244341,130.291883,53.347018,53.147416,0.259301,0.247869,130.849445,129.715555
17375,17375,0,129.261313,130.354561,53.646210,53.347018,0.281121,0.254519,130.871962,129.837038
17376,17376,0,129.281599,130.444603,54.336387,53.646210,0.322933,0.268202,131.071448,129.737552


In [19]:
concatenated_df.to_csv("./marked_data/result_with_parameters.csv", index=False)